# Geographical Names
Matching geographical names from the Ur III dataset with the [Pleiades](http://pleiades.stoa.org) data. Dataset of ancient place names downloaded in JSON from [Pleiades](https://pleiades.stoa.org/downloads).

In [1]:
import json
import requests
from tqdm.auto import tqdm
import pandas as pd
import zipfile

Download necessary files. Make sure there is a directory `jsonzip`.

In [ ]:
files = ["http://build-oracc.museum.upenn.edu/json/epsd2-admin-ur3.zip",
         "http://atlantides.org/downloads/pleiades/json/pleiades-places-latest.json.gz"]

In [ ]:
CHUNK = 1024
for f in files:
    url = f
    file = f'jsonzip/{f.split("/")[-1]}'
    with requests.get(url, stream=True) as r:
        if r.status_code == 200:
            total_size = int(r.headers.get('content-length', 0))
            tqdm.write(f'Saving {url} as {file}')
            t=tqdm(total=total_size, unit='B', unit_scale=True)
            with open(file, 'wb') as f:
                for c in r.iter_content(chunk_size=CHUNK):
                    t.update(len(c))
                    f.write(c)
        else:
            tqdm.write(f"WARNING: {url} does not exist.")

In [2]:
import gzip
f = gzip.open('jsonzip/pleiades-places-latest.json.gz', 'rb')
file_content = f.read()
f.close()

In [3]:
data =json.loads(file_content)

In [4]:
places = data['@graph']

Place names are taken from the field `romanized` in `names` - are there better fields to use?

In [5]:
placenames = {}
for p in places:
    coordinates = p.get("reprPoint")
    names = p.get("names")
    for n in names:
        name = n.get("romanized")
        d = {name: coordinates}
        placenames.update(d)

## Example 1: Drehem

In [6]:
placenames.get("Drehem")

[45.290576, 32.060569]

## Example 2: Puzriš-Dagan
This finds nothing. Why? Because the name is represented as "Puzriš-Dagan, Puzrish-Dagan, Puzurish-Dagan" as a single string.

In [7]:
placenames.get("Puzriš-Dagan")

In [8]:
placenames.get("Puzriš-Dagan, Puzrish-Dagan, Puzurish-Dagan")

[45.290576, 32.060569]

## Example 3: Ŋirsu
This finds nothing. Why? The dataset does not represent nasal G (Ŋ), but writes Girsu

In [9]:
placenames.get("Ŋirsu")

In [10]:
placenames.get("Girsu")

[46.175712, 31.576909]

## Example 4: Telloh (modern name for Ŋirsu/Girsu)
The full name in Pleiades is Tell Telloh, not Telloh.

In [11]:
placenames.get("Telloh")

In [12]:
placenames.get("Tell Telloh")

[46.175712, 31.576909]

## Example 5: Agade
Very different issue: coordinates are unknown.

In [13]:
placenames.get("Agade")

## Example 6: Irisaŋrig
The name is written " Iri-Saĝrig, Irisagrig, Urusagrig" - with a blank at the beginning! The location of the site is known, but not recorded yet in Pleiades.

In [14]:
placenames[" Iri-Saĝrig, Irisagrig, Urusagrig"]

In [15]:
def parsejson(text, id_text):
    l = []
    for JSONobject in text["cdl"]:
        if "cdl" in JSONobject: 
            l.extend(parsejson(JSONobject, id_text))
        if "f" in JSONobject:
            lemm = JSONobject["f"]
            lemm["id_text"] = id_text
            l.append(lemm)
    return l

In [16]:
lemm_l = [] # initiate the list that will hold all the lemmatization data of all texts
file = f"jsonzip/epsd2-admin-ur3.zip"
try:
    z = zipfile.ZipFile(file)       # create a Zipfile object
except:
    print(f"{file} does not exist or is not a proper ZIP file")
files = z.namelist()     # list of all the files in the ZIP
files = [name for name in files if "corpusjson" in name and name[-5:] == '.json']                                                                                                  #that holds all the P, Q, and X numbers.
for filename in tqdm(files):                            #iterate over the file names
    id_text = filename[-13:-5] # id_text is, for instance, blms/P414332
    try:
        text = z.read(filename).decode('utf-8')         #read and decode the json file of one particular text
        data_json = json.loads(text)                # make it into a json object (essentially a dictionary)
        lemm_l.extend(parsejson(data_json, id_text))               # and send to the parsejson() function
    except:
        tqdm.write(f'{id_text} is not available or not complete')
z.close()

In [17]:
words = pd.DataFrame(lemm_l)
keep = ["id_text", "cf", "gw", "pos"]
words = words[keep]
words = words.fillna("")
GeographicalPOS = ["SN", "GN", "WN"]
words = words.loc[words.pos.isin(GeographicalPOS)]
GeographicalNames = set(words["cf"])

In [18]:
Coordinates = {}
for name in GeographicalNames:
    location = placenames.get(name)
    if location: 
        c = {name: location}
        Coordinates.update(c)

A literal search finds 28 out of 1937 geographical names (this is for the entire Ur III data set, not just for Drehem). Taking into account the issues discussed above may increase that number.

In [19]:
len(GeographicalNames), len(Coordinates)

(1937, 28)

In [20]:
Coordinates

{'Mari': [40.8843621, 34.5523591],
 'Unu': [31.284786, 30.131757],
 'Unug': [45.61715, 31.318603],
 'Tutub': [44.5576922, 33.3574222],
 'La': [22.498245, 36.725537],
 'Uruk': [45.61715, 31.318603],
 'Gaba': [35.106902, 32.730365],
 'Zabalam': [45.87799811166091, 31.743104673931537],
 'Uršu': [37.25, 37.25],
 'Larsam': [45.853611, 31.285833],
 'Nibru': [45.230833, 32.126944],
 'Karkar': [45.725183, 31.815617],
 'Keš': [45.660285717336144, 32.31245585941179],
 'Adab': [45.62585625145, 31.9512722901],
 'Sirara': [35.8575, 33.416111],
 'Martu': [41.027992, 34.389685],
 'Marad': [44.7860122, 32.0711659],
 'Ebla': [36.798365802707416, 35.79904467871221],
 'Isin': [45.283333, 31.933332],
 'Elam': [49.5, 31.5],
 'Buranun': [39.606017877774164, 35.54331017895074],
 'Gubla': [35.646288999999996, 34.1192635],
 'Umma': [45.9, 31.65],
 'Dabrum': [45.725183, 31.815617],
 'Murum': [-3.360099, 39.181617],
 'Dilmun': [49.21475845511888, 27.760627439275623],
 'Aššur': [43.26224277729484, 35.453921526620

In [21]:
placenames2 = placenames.copy()
for name in placenames: 
    coordinates = placenames.get(name)
    names = name.split(',')
    n_c = {n.strip():coordinates for n in names}
    for n in names:
        n1 = n.strip().replace('g̃', 'ŋ')
        n_c.update({n1 : coordinates})
        n2 = n1.replace('G̃', 'Ŋ')
        n_c.update({n2 : coordinates})
        n3 = n2.replace('-', '')
        n_c.update({n3 : coordinates})
        n4 = n3.replace('G', 'Ŋ')
        n_c.update({n4 : coordinates})
        n5 = n4.replace('g', 'ŋ')
        n_c.update({n5 : coordinates})
    placenames2.update(n_c)

In [22]:
len(placenames), len(placenames2)

(30367, 43133)

In [23]:
Coordinates2 = {}
for name in GeographicalNames:
    location = placenames2.get(name)
    if location: 
        c = {name: location}
        Coordinates2.update(c)

In [24]:
Coordinates2

{'': [83.2121, 53.4517],
 'Mari': [40.8843621, 34.5523591],
 'Unu': [45.61715, 31.318603],
 'Unug': [45.61715, 31.318603],
 'Tutub': [44.5576922, 33.3574222],
 'La': [22.498245, 36.725537],
 'Uruk': [45.61715, 31.318603],
 'Uru': [46.104748, 30.961581],
 'Gaba': [35.106902, 32.730365],
 'Zabalam': [45.87799811166091, 31.743104673931537],
 'Lagaš': [46.40836, 31.420485],
 'Urim': [46.104748, 30.961581],
 'Uršu': [37.25, 37.25],
 'Larsam': [45.853611, 31.285833],
 'Ninua': [43.155403, 36.366841],
 'Nibru': [45.230833, 32.126944],
 'Šuruppak': [45.5097359, 31.7778493],
 'Karkar': [45.725183, 31.815617],
 'Keš': [45.660285717336144, 32.31245585941179],
 'Adab': [45.62585625145, 31.9512722901],
 'Sirara': [35.8575, 33.416111],
 'Ŋirsu': [46.175712, 31.576909],
 'Martu': [41.027992, 34.389685],
 'Marad': [44.7860122, 32.0711659],
 'Arraphum': [44.3953, 35.468099],
 'PuzrišDagan': [45.290576, 32.060569],
 'Ebla': [36.798365802707416, 35.79904467871221],
 'Isin': [45.283333, 31.933332],
 'Elam

In [28]:
placenames2.get("Simanum")

In [32]:
placenames2["Irisagrig"]